In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from concurrent.futures import ThreadPoolExecutor
import pickle
    
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Activation, Add,
    MaxPooling2D, GlobalAveragePooling2D, Dense,
    Dropout, Input
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, accuracy_score,
    confusion_matrix, mean_squared_error
)

from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import resample
# Reproducibility
np.random.seed(42)
tf.random.set_seed(42)



In [2]:
dir_images = r'C:\Users\patel\OneDrive\Desktop\FPR\images'
file_meta  = r'C:\Users\patel\OneDrive\Desktop\FPR\myntradataset\styles.csv'

In [3]:
df_full = pd.read_csv(file_meta,on_bad_lines='skip')
df_full['id'] = df_full['id'].astype(str)

# List images and intersect
files = [f for f in os.listdir(dir_images)
         if f.lower().endswith(('.jpg','.png','.jpeg'))]
ids_all = [os.path.splitext(f)[0] for f in files]
common = list(set(ids_all) & set(df_full['id']))
print(f"Found {len(common)} images with metadata.")

# Sample 1,000 IDs
sample_ids = np.random.choice(common, size=25000, replace=False)
df = df_full[df_full['id'].isin(sample_ids)].reset_index(drop=True)
ids = df['id'].tolist()
print(f"Sampling {len(ids)} images for modeling.")

# Attributes
attrs = ['gender','masterCategory','subCategory',
         'articleType','baseColour','usage']

Found 44419 images with metadata.
Sampling 25000 images for modeling.


In [ ]:
enc_dir = 'label_encoders'
os.makedirs(enc_dir, exist_ok=True)
label_encoders = {}
num_classes = {}
y_int = {}
for attr in attrs:
    codes, uniques = pd.factorize(df[attr])
    codes
    label_encoders[attr] = uniques
    num_classes[attr] = len(uniques)
    y_int[attr] = codes

# One-hot
Y_oh = {attr: to_categorical(y_int[attr], num_classes[attr])
        for attr in attrs}


In [27]:
import pandas as pd

# Assuming you’ve already run your factorization:
# label_encoders[attr] = uniques
# y_int[attr] = codes

for attr in attrs:
    uniques = label_encoders[attr]
    codes = list(range(len(uniques)))
    df_map = pd.DataFrame({
        'Encoded': codes,
        'Original': uniques
    })
    print(f"\n=== {attr} Encoding Map ===")
    print(df_map.to_markdown(index=False))



=== gender Encoding Map ===
|   Encoded | Original   |
|----------:|:-----------|
|         0 | Men        |
|         1 | Women      |
|         2 | Boys       |
|         3 | Girls      |
|         4 | Unisex     |

=== masterCategory Encoding Map ===
|   Encoded | Original       |
|----------:|:---------------|
|         0 | Apparel        |
|         1 | Accessories    |
|         2 | Footwear       |
|         3 | Personal Care  |
|         4 | Free Items     |
|         5 | Sporting Goods |
|         6 | Home           |

=== subCategory Encoding Map ===
|   Encoded | Original                 |
|----------:|:-------------------------|
|         0 | Bottomwear               |
|         1 | Topwear                  |
|         2 | Socks                    |
|         3 | Watches                  |
|         4 | Shoes                    |
|         5 | Belts                    |
|         6 | Flip Flops               |
|         7 | Bags                     |
|         8 | Innerwea

In [5]:
IMG_SIZE = (128, 128)
NUM_THREADS = 8  # You can tune this based on your CPU

def load_img(id_):
    try:
        img_path = os.path.join(dir_images, f"{id_}.jpg")
        img = Image.open(img_path).convert('RGB').resize(IMG_SIZE)
        return np.array(img) / 255.0
    except Exception as e:
        print(f"Error loading image {id_}: {e}")
        return np.zeros((*IMG_SIZE, 3))  # Fallback to blank image if error

# --- Parallel image loading ---
with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
    X = np.array(list(executor.map(load_img, ids)))

# --- Labels ---
Y = [Y_oh[attr] for attr in attrs]

# --- Output Shapes ---
print("X shape:", X.shape)
for attr in attrs:
    print(f"{attr} one-hot shape:", Y_oh[attr].shape)

X shape: (25000, 128, 128, 3)
gender one-hot shape: (25000, 5)
masterCategory one-hot shape: (25000, 7)
subCategory one-hot shape: (25000, 45)
articleType one-hot shape: (25000, 139)
baseColour one-hot shape: (25000, 46)
usage one-hot shape: (25000, 8)


In [6]:
idx = np.arange(len(X))
i_tr, i_te = train_test_split(idx, test_size=0.2, random_state=42)
X_tr, X_te = X[i_tr], X[i_te]
y_tr = [Y[k][i_tr] for k in range(len(attrs))]
y_te = [Y[k][i_te] for k in range(len(attrs))]
y_te_int = [y_int[attr][i_te] for attr in attrs]
print("Train/Test shapes:", X_tr.shape, X_te.shape)

Train/Test shapes: (20000, 128, 128, 3) (5000, 128, 128, 3)


In [7]:
def build_model_complex(input_shape, attrs, num_classes, wd=1e-4):
    def conv_block(x, filters):
        x = Conv2D(filters, 3, padding='same', kernel_regularizer=l2(wd))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        return x

    def res_block(x, filters):
        shortcut = x
        x = conv_block(x, filters)
        x = conv_block(x, filters)
        if shortcut.shape[-1] != filters:
            shortcut = Conv2D(filters,1,padding='same',kernel_regularizer=l2(wd))(shortcut)
            shortcut = BatchNormalization()(shortcut)
        x = Add()([x, shortcut])
        return Activation('relu')(x)

    inp = Input(shape=input_shape)
    x = conv_block(inp, 32)
    x = MaxPooling2D()(x)
    x = res_block(x, 64)
    x = MaxPooling2D()(x)
    x = res_block(x, 128)
    x = MaxPooling2D()(x)
    x = res_block(x, 256)
    x = MaxPooling2D()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(wd))(x)
    x = Dropout(0.5)(x)
    outs = [Dense(num_classes[a], activation='softmax',
                 name=a, kernel_regularizer=l2(wd))(x)
            for a in attrs]
    m = Model(inp, outs)
    m.compile(
        optimizer=Adam(1e-3),
        loss={a:'categorical_crossentropy' for a in attrs},
        metrics={a:['accuracy'] for a in attrs}
    )
    return m


In [8]:
import numpy as np

batch_sizes = [16, 32, 64]
epochs = 10
histories = {}
models = {}

best_acc = 0.0
best_bs = None

for bs in batch_sizes:
    print(f"Training bs={bs}")
    model = build_model_complex((*IMG_SIZE, 3), attrs, num_classes)
    hist = model.fit(
        X_tr,
        {a: y_tr[i] for i, a in enumerate(attrs)},
        validation_data=(X_te, {a: y_te[i] for i, a in enumerate(attrs)}),
        epochs=epochs,
        batch_size=bs,
        verbose=1,
    )
    histories[bs] = hist
    models[bs] = model

    
    final_epoch = epochs - 1
    val_accs = [hist.history[f'val_{a}_accuracy'][final_epoch] for a in attrs]
    mean_val_acc = np.mean(val_accs)
    print(f"Mean val accuracy for bs={bs}: {mean_val_acc:.4f}")

    if mean_val_acc > best_acc:
        best_acc = mean_val_acc
        best_bs = bs


Training bs=16
Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 489s 385ms/step - articleType_accuracy: 0.2806 - articleType_loss: 2.9636 - baseColour_accuracy: 0.3544 - baseColour_loss: 2.4398 - gender_accuracy: 0.6104 - gender_loss: 0.9385 - loss: 9.6044 - masterCategory_accuracy: 0.7669 - masterCategory_loss: 0.6192 - subCategory_accuracy: 0.5709 - subCategory_loss: 1.6569 - usage_accuracy: 0.7631 - usage_loss: 0.7979 - val_articleType_accuracy: 0.2568 - val_articleType_loss: 2.9651 - val_baseColour_accuracy: 0.3404 - val_baseColour_loss: 2.4236 - val_gender_accuracy: 0.5880 - val_gender_loss: 0.9401 - val_loss: 10.1907 - val_masterCategory_accuracy: 0.7970 - val_masterCategory_loss: 0.5926 - val_subCategory_accuracy: 0.3866 - val_subCategory_loss: 2.1908 - val_usage_accuracy: 0.7742 - val_usage_loss: 0.8295
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 442s 354ms/step - articleType_accuracy: 0.5232 - articleType_loss: 1.6925 - baseColour_accuracy: 0.4503 - baseColour_loss: 1.9535 - gender_acc

In [9]:
# After loop, save the best model
print(f"\nBest batch size: {best_bs} with mean val accuracy {best_acc:.4f}")
best_model = models[best_bs]

# 1) Ensure the right folder
os.makedirs('models4', exist_ok=True)

# 2) Build a valid filepath with a .keras extension
filepath = f'models4/best_model_bs{best_bs}.h5'

# 3) Save in native Keras format
best_model.save(filepath)

print(f"Saved best model to {filepath}")



Best batch size: 16 with mean val accuracy 0.7789
Saved best model to models4/best_model_bs16.h5


In [10]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Assumes: batch_sizes, epochs, attrs, histories are already defined

# 1) Ensure output folder exists
os.makedirs('model_plots', exist_ok=True)

# 2) Create the figure
fig, axes = plt.subplots(1, len(batch_sizes), figsize=(18, 5), sharey=True)

for ax, bs in zip(axes, batch_sizes):
    ax.set_title(f"Batch size = {bs}")
    ax.set_xlabel("Epoch")
    if ax is axes[0]:
        ax.set_ylabel("Validation Accuracy")
    
    # Plot each attribute's validation accuracy
    for attr in attrs:
        acc_vals = histories[bs].history[f'val_{attr}_accuracy']
        ax.plot(range(1, epochs+1), acc_vals, marker='o', label=attr)
        
    # Plot mean accuracy
    mean_vals = [
        np.mean([histories[bs].history[f'val_{a}_accuracy'][e] for a in attrs])
        for e in range(epochs)
    ]
    ax.plot(
        range(1, epochs+1),
        mean_vals,
        marker='s',
        linewidth=2,
        color='black',
        label='mean'
    )
    # Annotate mean values
    for e, v in enumerate(mean_vals, start=1):
        ax.text(e, v - 0.005, f"{v:.2f}", ha='center', va='top', fontsize=8)

    ax.set_xticks(range(1, epochs+1))
    ax.legend(loc='lower right', fontsize='small')

plt.tight_layout()

# 3) Save the figure
output_path = 'model_plots/validation_accuracy_by_batch_size.png'
fig.savefig(output_path, dpi=300, bbox_inches='tight')
plt.close(fig)

print(f"Plot saved to {output_path}")
plt.tight_layout()
plt.show()

Plot saved to model_plots/validation_accuracy_by_batch_size.png


<Figure size 640x480 with 0 Axes>

In [11]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1) First, melt your histories into a tidy DataFrame
records = []
for bs in batch_sizes:
    h = histories[bs].history
    for epoch in range(1, epochs + 1):
        for attr in attrs:
            records.append({
                'batch_size': bs,
                'epoch': epoch,
                'attribute': attr,
                'loss':    h[f'{attr}_loss'][epoch - 1],
                'type':    'train'
            })
            records.append({
                'batch_size': bs,
                'epoch': epoch,
                'attribute': attr,
                'loss':    h[f'val_{attr}_loss'][epoch - 1],
                'type':    'val'
            })
df_loss = pd.DataFrame(records)

# 2) Ensure output folder exists
os.makedirs('model_plots', exist_ok=True)

# 3) Draw facets: rows by attribute, col by train/val
g = sns.FacetGrid(
    df_loss,
    row='attribute',
    col='type',
    hue='batch_size',
    sharey=False,      # or True if you want same scale
    height=2.5,
    aspect=2
)
g.map(sns.lineplot, 'epoch', 'loss')
g.add_legend(title='Batch Size')
g.set_axis_labels('Epoch', 'Loss')
g.set_titles(row_template='{row_name}', col_template='{col_name}')
plt.subplots_adjust(hspace=0.3)

# 4) Save the figure
output_path = 'model_plots/loss_by_attribute_and_type.png'
g.fig.savefig(output_path, dpi=300, bbox_inches='tight')
plt.close(g.fig)

print(f"Plot saved to {output_path}")
plt.tight_layout()
plt.show()

Plot saved to model_plots/loss_by_attribute_and_type.png


<Figure size 640x480 with 0 Axes>

In [12]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1) Melt histories into a tidy DataFrame for accuracy
records = []
for bs in batch_sizes:
    h = histories[bs].history
    for epoch in range(1, epochs + 1):
        for attr in attrs:
            records.append({
                'batch_size': bs,
                'epoch': epoch,
                'attribute': attr,
                'accuracy': h[f'{attr}_accuracy'][epoch - 1],
                'type': 'train'
            })
            records.append({
                'batch_size': bs,
                'epoch': epoch,
                'attribute': attr,
                'accuracy': h[f'val_{attr}_accuracy'][epoch - 1],
                'type': 'val'
            })
df_acc = pd.DataFrame(records)

# 2) Ensure output folder exists
os.makedirs('model_plots', exist_ok=True)

# 3) Facet by attribute (rows) and type (cols), hue by batch size
g = sns.FacetGrid(
    df_acc,
    row='attribute',
    col='type',
    hue='batch_size',
    sharey=False,
    height=2.5,
    aspect=2
)
g.map(sns.lineplot, 'epoch', 'accuracy', marker='o')
g.add_legend(title='Batch Size')
g.set_axis_labels('Epoch', 'Accuracy')
g.set_titles(row_template='{row_name}', col_template='{col_name}')
plt.subplots_adjust(hspace=0.3)

# 4) Save the figure
output_path = 'model_plots/accuracy_by_attribute_and_type.png'
g.fig.savefig(output_path, dpi=300, bbox_inches='tight')
plt.close(g.fig)

print(f"Plot saved to {output_path}")
plt.tight_layout()
plt.show()

Plot saved to model_plots/accuracy_by_attribute_and_type.png


<Figure size 640x480 with 0 Axes>

In [13]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

# 1) Ensure output folder exists
os.makedirs('model_reports', exist_ok=True)

# 2) Predict on test set
y_pred = best_model.predict(X_te)

# 3) Generate and save classification reports & accuracy
df_reports = {}
acc_per_attr = {}

for i, a in enumerate(attrs):
    # a) Convert predictions and true labels to integer class indices
    y_p = np.argmax(y_pred[i], axis=1)
    y_t = y_te_int[i]
    
    # b) Build the report as a DataFrame
    report_dict = classification_report(
        y_t, y_p,
        labels=list(range(num_classes[a])),
        target_names=label_encoders[a],
        output_dict=True
    )
    df_rep = pd.DataFrame(report_dict).T
    df_reports[a] = df_rep
    
    # c) Compute overall accuracy
    acc = accuracy_score(y_t, y_p)
    acc_per_attr[a] = acc
    
    # d) Print to console
    print(f"\n=== {a} ===\n", classification_report(
        y_t, y_p,
        labels=list(range(num_classes[a])),
        target_names=label_encoders[a]
    ))
    
    # e) Save the report DataFrame to CSV
    csv_path = f'model_reports/classification_report_{a}.csv'
    df_rep.to_csv(csv_path, index=True)
    print(f"Saved classification report for '{a}' to {csv_path}")

# 4) Save summary of accuracies
df_acc_summary = pd.DataFrame.from_dict(
    acc_per_attr, orient='index', columns=['accuracy']
)
summary_path = 'model_reports/accuracy_summary.csv'
df_acc_summary.to_csv(summary_path)
print(f"\nSaved accuracy summary to {summary_path}")


157/157 ━━━━━━━━━━━━━━━━━━━━ 25s 156ms/step

=== gender ===
               precision    recall  f1-score   support

         Men       0.81      0.95      0.88      2492
       Women       0.89      0.82      0.85      2122
        Boys       0.00      0.00      0.00        85
       Girls       0.25      0.03      0.05        65
      Unisex       0.64      0.36      0.46       236

    accuracy                           0.84      5000
   macro avg       0.52      0.43      0.45      5000
weighted avg       0.82      0.84      0.82      5000

Saved classification report for 'gender' to model_reports/classification_report_gender.csv

=== masterCategory ===
                 precision    recall  f1-score   support

       Apparel       0.99      0.93      0.96      2436
   Accessories       0.81      0.99      0.89      1233
      Footwear       1.00      0.97      0.98      1069
 Personal Care       0.95      0.72      0.82       246
    Free Items       0.00      0.00      0.00        

c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [14]:
from sklearn.metrics import classification_report, accuracy_score

# Assuming: attrs, y_te_int, y_pred, label_encoders, num_classes are defined

for attr in attrs:
    i = attrs.index(attr)
    y_true = y_te_int[i]
    y_pred_cls = np.argmax(y_pred[i], axis=1)
    
    print(f"\n=== Classification Report: {attr} ===\n")
    print(classification_report(
        y_true,
        y_pred_cls,
        labels=list(range(num_classes[attr])),
        target_names=label_encoders[attr]
    ))
    print(f"Accuracy for {attr}: {accuracy_score(y_true, y_pred_cls):.4f}")



=== Classification Report: gender ===

              precision    recall  f1-score   support

         Men       0.81      0.95      0.88      2492
       Women       0.89      0.82      0.85      2122
        Boys       0.00      0.00      0.00        85
       Girls       0.25      0.03      0.05        65
      Unisex       0.64      0.36      0.46       236

    accuracy                           0.84      5000
   macro avg       0.52      0.43      0.45      5000
weighted avg       0.82      0.84      0.82      5000

Accuracy for gender: 0.8374

=== Classification Report: masterCategory ===

                precision    recall  f1-score   support

       Apparel       0.99      0.93      0.96      2436
   Accessories       0.81      0.99      0.89      1233
      Footwear       1.00      0.97      0.98      1069
 Personal Care       0.95      0.72      0.82       246
    Free Items       0.00      0.00      0.00        14
Sporting Goods       0.00      0.00      0.00         1
  

c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [15]:
import os
import numpy as np
from sklearn.metrics import classification_report, accuracy_score

# 1) Ensure output folder exists
os.makedirs('model_reports', exist_ok=True)

# 2) Loop through each attribute, print and save reports
accuracy_summary = {}

for attr in attrs:
    i = attrs.index(attr)
    y_true = y_te_int[i]
    y_pred_cls = np.argmax(y_pred[i], axis=1)
    
    # a) Generate the text report
    report_text = classification_report(
        y_true,
        y_pred_cls,
        labels=list(range(num_classes[attr])),
        target_names=label_encoders[attr]
    )
    accuracy = accuracy_score(y_true, y_pred_cls)
    accuracy_summary[attr] = accuracy
    
    # b) Print to console
    print(f"\n=== Classification Report: {attr} ===\n")
    print(report_text)
    print(f"Accuracy for {attr}: {accuracy:.4f}")
    
    # c) Save the report to a .txt file
    txt_path = f'model_reports/classification_report_{attr}.txt'
    with open(txt_path, 'w') as f:
        f.write(f"Classification Report for '{attr}'\n\n")
        f.write(report_text)
        f.write(f"\nAccuracy: {accuracy:.4f}\n")
    print(f"Saved report to {txt_path}")

# 3) Save overall accuracies to CSV
import pandas as pd

df_acc_summary = pd.DataFrame.from_dict(
    accuracy_summary, orient='index', columns=['accuracy']
)
csv_path = 'model_reports/accuracy_summary.csv'
df_acc_summary.to_csv(csv_path, index_label='attribute')
print(f"\nSaved accuracy summary to {csv_path}")



=== Classification Report: gender ===

              precision    recall  f1-score   support

         Men       0.81      0.95      0.88      2492
       Women       0.89      0.82      0.85      2122
        Boys       0.00      0.00      0.00        85
       Girls       0.25      0.03      0.05        65
      Unisex       0.64      0.36      0.46       236

    accuracy                           0.84      5000
   macro avg       0.52      0.43      0.45      5000
weighted avg       0.82      0.84      0.82      5000

Accuracy for gender: 0.8374
Saved report to model_reports/classification_report_gender.txt

=== Classification Report: masterCategory ===

                precision    recall  f1-score   support

       Apparel       0.99      0.93      0.96      2436
   Accessories       0.81      0.99      0.89      1233
      Footwear       1.00      0.97      0.98      1069
 Personal Care       0.95      0.72      0.82       246
    Free Items       0.00      0.00      0.00      

c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [16]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report

# 1) Ensure output folder exists
os.makedirs('model_reports', exist_ok=True)

# 2) Per-attribute reports and accuracies
accuracy_summary = {}
for i, attr in enumerate(attrs):
    y_true = y_te_int[i]
    y_pred_cls = np.argmax(y_pred[i], axis=1)
    
    # a) Text report
    report_text = classification_report(
        y_true,
        y_pred_cls,
        labels=list(range(num_classes[attr])),
        target_names=label_encoders[attr]
    )
    acc = accuracy_score(y_true, y_pred_cls)
    accuracy_summary[attr] = acc
    
    # b) Print to console
    print(f"\n--- {attr} ---")
    print(report_text)
    print(f"Accuracy ({attr}): {acc:.4f}")
    
    # c) Save to file
    txt_path = f'model_reports/classification_report_{attr}.txt'
    with open(txt_path, 'w') as f:
        f.write(f"=== Classification Report: {attr} ===\n\n")
        f.write(report_text)
        f.write(f"\nAccuracy: {acc:.4f}\n")
    print(f"Saved report for '{attr}' to {txt_path}")

# 3) Mean accuracy across all attributes
mean_acc = np.mean(list(accuracy_summary.values()))
print(f"\nMean tag accuracy: {mean_acc:.4f}")

# Save summary of per-attribute accuracies
df_acc = pd.DataFrame.from_dict(accuracy_summary, orient='index', columns=['accuracy'])
df_acc.index.name = 'attribute'
summary_path = 'model_reports/accuracy_summary.csv'
df_acc.to_csv(summary_path)
print(f"Saved accuracy summary to {summary_path}")

# 4) Flattened overall accuracy
y_true_flat = np.concatenate([y_te_int[i] for i in range(len(attrs))])
y_pred_flat = np.concatenate([np.argmax(y_pred[i], axis=1) for i in range(len(attrs))])
overall_flat_acc = accuracy_score(y_true_flat, y_pred_flat)
print(f"Overall (flattened) accuracy: {overall_flat_acc:.4f}")

# 5) Combined classification report (flattened)
# Build global label names
global_labels = []
for attr in attrs:
    global_labels += [f"{attr}_{cls}" for cls in label_encoders[attr]]

combined_report = classification_report(
    y_true_flat,
    y_pred_flat,
    labels=list(range(len(global_labels))),
    target_names=global_labels
)

print("\n=== Combined Classification Report (Flattened) ===")
print(combined_report)

# Save combined report to file
combined_path = 'model_reports/combined_classification_report.txt'
with open(combined_path, 'w') as f:
    f.write("=== Combined Classification Report (Flattened) ===\n\n")
    f.write(combined_report)
    f.write(f"\nOverall (flattened) accuracy: {overall_flat_acc:.4f}\n")
print(f"Saved combined report to {combined_path}")



--- gender ---
              precision    recall  f1-score   support

         Men       0.81      0.95      0.88      2492
       Women       0.89      0.82      0.85      2122
        Boys       0.00      0.00      0.00        85
       Girls       0.25      0.03      0.05        65
      Unisex       0.64      0.36      0.46       236

    accuracy                           0.84      5000
   macro avg       0.52      0.43      0.45      5000
weighted avg       0.82      0.84      0.82      5000

Accuracy (gender): 0.8374
Saved report for 'gender' to model_reports/classification_report_gender.txt

--- masterCategory ---
                precision    recall  f1-score   support

       Apparel       0.99      0.93      0.96      2436
   Accessories       0.81      0.99      0.89      1233
      Footwear       1.00      0.97      0.98      1069
 Personal Care       0.95      0.72      0.82       246
    Free Items       0.00      0.00      0.00        14
Sporting Goods       0.00      0

c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,


=== Combined Classification Report (Flattened) ===
                                       precision    recall  f1-score   support

                           gender_Men       0.85      0.94      0.89     10174
                         gender_Women       0.89      0.82      0.85      6568
                          gender_Boys       0.76      0.74      0.75      2343
                         gender_Girls       0.81      0.62      0.70      1115
                        gender_Unisex       0.76      0.83      0.79      1943
               masterCategory_Apparel       0.64      0.71      0.67      1071
           masterCategory_Accessories       0.60      0.54      0.57       260
              masterCategory_Footwear       0.85      0.76      0.80       458
         masterCategory_Personal Care       0.81      0.67      0.73       827
            masterCategory_Free Items       0.46      0.57      0.51       209
        masterCategory_Sporting Goods       0.58      0.67      0.62       406

c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

# 1) Compute overall MSE & RMSE
Yt = np.concatenate([y_te[i] for i in range(len(attrs))], axis=1)
Yp = np.concatenate([y_pred[i] for i in range(len(attrs))], axis=1)
mse = mean_squared_error(Yt, Yp)
rmse = np.sqrt(mse)
print(f"Overall MSE: {mse:.4f}, RMSE: {rmse:.4f}")

# 2) Ensure output folder exists
os.makedirs('model_plots', exist_ok=True)

# 3) Plot MSE & RMSE as a bar chart
fig, ax = plt.subplots(figsize=(6, 4))
ax.bar(['MSE', 'RMSE'], [mse, rmse])
ax.set_title('Overall MSE & RMSE')
ax.set_ylabel('Value')
for i, v in enumerate([mse, rmse]):
    ax.text(i, v + (v * 0.02), f"{v:.2f}", ha='center', va='bottom', fontsize=10)

plt.tight_layout()

# 4) Save the figure
output_path = 'model_plots/mse_rmse_overall.png'
fig.savefig(output_path, dpi=300, bbox_inches='tight')
plt.close(fig)

print(f"Plot saved to {output_path}")
plt.tight_layout()
plt.show()

Overall MSE: 0.0074, RMSE: 0.0859
Plot saved to model_plots/mse_rmse_overall.png


<Figure size 640x480 with 0 Axes>

In [18]:
import os
import numpy as np
import matplotlib.pyplot as plt

# 1) Select history for batch size 32
hist = histories[32].history  # replace 32 with whichever batch size you want

# 2) Ensure output folder exists
os.makedirs('model_plots', exist_ok=True)

# 3) Create the figure with two subplots
fig, axes = plt.subplots(2, 1, figsize=(8, 8))

# (a) Total Loss plot
axes[0].plot(hist['loss'], label='train total loss')
axes[0].plot(hist['val_loss'], label='val total loss')
axes[0].set_title('Total Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend(loc='upper right')

# (b) Mean Accuracy Across Tags
avg_train_acc = [
    np.mean([hist[f'{a}_accuracy'][i] for a in attrs])
    for i in range(len(hist['loss']))
]
avg_val_acc = [
    np.mean([hist[f'val_{a}_accuracy'][i] for a in attrs])
    for i in range(len(hist['loss']))
]
axes[1].plot(avg_train_acc, label='train mean acc', marker='o')
axes[1].plot(avg_val_acc,   label='val mean acc', marker='o')
axes[1].set_title('Mean Accuracy Across Tags')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend(loc='lower right')

plt.tight_layout()

# 4) Save the figure
output_path = 'model_plots/loss_and_mean_accuracy_bs32.png'
fig.savefig(output_path, dpi=300, bbox_inches='tight')
plt.close(fig)

print(f"Plot saved to {output_path}")
plt.tight_layout()
plt.show()

Plot saved to model_plots/loss_and_mean_accuracy_bs32.png


<Figure size 640x480 with 0 Axes>

In [19]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score
from tensorflow.keras.utils import to_categorical

# 1) Build big arrays of shape (N_total, )
y_true_list = []
y_score_list = []

for i, attr in enumerate(attrs):
    # one-hot true labels and predicted scores
    y_true = to_categorical(y_te_int[i], num_classes[attr])  # shape (n_samples, n_classes)
    y_score = y_pred[i]                                        # shape (n_samples, n_classes)
    
    # append flattened
    y_true_list.append(y_true.ravel())
    y_score_list.append(y_score.ravel())

# concatenate across attributes
y_true_all = np.concatenate(y_true_list)
y_score_all = np.concatenate(y_score_list)

# 2) Compute overall PR curve
prec, rec, _ = precision_recall_curve(y_true_all, y_score_all)
ap_micro = average_precision_score(y_true_all, y_score_all, average='micro')

# 3) Ensure output folder exists
os.makedirs('model_plots', exist_ok=True)

# 4) Plot
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(rec, prec, label=f'Micro-averaged PR (AP = {ap_micro:.2f})', marker='.')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('Overall (Micro-averaged) Precision–Recall Curve')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.legend(loc='lower left')
plt.tight_layout()

# 5) Save the figure
output_path = 'model_plots/precision_recall_curve_micro_avg.png'
fig.savefig(output_path, dpi=300, bbox_inches='tight')
plt.close(fig)

print(f"Precision-Recall curve saved to {output_path}")
plt.tight_layout()
plt.show()

Precision-Recall curve saved to model_plots/precision_recall_curve_micro_avg.png


<Figure size 640x480 with 0 Axes>

In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score
from tensorflow.keras.utils import to_categorical

# 1) Concatenate all one-hot true labels and predicted scores
y_true_all = np.concatenate([
    to_categorical(y_te_int[i], num_classes[attr])
    for i, attr in enumerate(attrs)
], axis=1)

y_score_all = np.concatenate([
    y_pred[i]
    for i in range(len(attrs))
], axis=1)

# 2) Compute micro-average precision–recall
prec, rec, _ = precision_recall_curve(y_true_all.ravel(), y_score_all.ravel())
ap_micro = average_precision_score(y_true_all, y_score_all, average='micro')

# 3) Ensure output folder exists
os.makedirs('model_plots', exist_ok=True)

# 4) Plot
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot(rec, prec, lw=2, label=f'Micro-avg PR (AP={ap_micro:.2f})')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('Overall Precision–Recall Curve (Micro-Average)')
ax.legend(loc='upper right', borderaxespad=0)
ax.grid(alpha=0.3)
plt.tight_layout()

# 5) Save the figure
output_path = 'model_plots/precision_recall_curve_micro_avg.png'
fig.savefig(output_path, dpi=300, bbox_inches='tight')
plt.close(fig)

print(f"Precision–Recall curve saved to {output_path}")
plt.tight_layout()
plt.show()

Precision–Recall curve saved to model_plots/precision_recall_curve_micro_avg.png


<Figure size 640x480 with 0 Axes>

In [21]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.utils import to_categorical

# 1) Choose attribute index (e.g., first attribute)
i = 0
attr = attrs[i]

# 2) Prepare true labels and predicted scores
y_true = to_categorical(y_te_int[i], num_classes[attr])  # shape (n_samples, n_classes)
y_score = y_pred[i]                                       # shape (n_samples, n_classes)

# 3) Ensure output folder exists
os.makedirs('model_plots', exist_ok=True)

# 4) Plot ROC curves
fig, ax = plt.subplots(figsize=(6, 6))
for c in range(num_classes[attr]):
    fpr, tpr, _ = roc_curve(y_true[:, c], y_score[:, c])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, label=f'{label_encoders[attr][c]} (AUC={roc_auc:.2f})')

# Diagonal line
ax.plot([0, 1], [0, 1], 'k--', linewidth=1)

# Labels and title
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title(f'ROC Curve for {attr}')
ax.legend(loc='lower right', fontsize='small')
ax.grid(alpha=0.3)

plt.tight_layout()

# 5) Save the figure
output_path = f'model_plots/roc_curve_{attr}.png'
fig.savefig(output_path, dpi=300, bbox_inches='tight')
plt.close(fig)

print(f"ROC curve saved to {output_path}")
plt.tight_layout()
plt.show()

ROC curve saved to model_plots/roc_curve_gender.png


<Figure size 640x480 with 0 Axes>

In [22]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

# 1) Compute per-attribute macro-F1
macro_f1s = []
for i, attr in enumerate(attrs):
    y_true        = y_te_int[i]
    y_pred_labels = np.argmax(y_pred[i], axis=1)
    macro = f1_score(y_true, y_pred_labels, average='macro', zero_division=0)
    macro_f1s.append(macro)

# 2) Compute overall F1
overall_f1 = np.mean(macro_f1s)

# 3) Ensure output folder exists
os.makedirs('model_plots', exist_ok=True)

# 4) Plot Overall F1
fig, ax = plt.subplots(figsize=(5, 5))
ax.bar(['Overall F1'], [overall_f1])
ax.set_ylim(0, 1)
ax.set_title('Overall Model F1 Score')
ax.set_ylabel('F1 Score')

# Annotate the bar
ax.text(0, overall_f1 + 0.02, f'{overall_f1:.2f}', ha='center', va='bottom', fontsize=12)

plt.tight_layout()

# 5) Save the figure
output_path = 'model_plots/overall_f1_score.png'
fig.savefig(output_path, dpi=300, bbox_inches='tight')
plt.close(fig)

print(f"Overall F1 score plot saved to {output_path}")
plt.tight_layout()
plt.show()

Overall F1 score plot saved to model_plots/overall_f1_score.png


<Figure size 640x480 with 0 Axes>

In [23]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tensorflow.keras.models import Model

# 1) Extract penultimate‐layer output
penult_layer = best_model.layers[-2].output  # second‐to‐last layer
feat_extractor = Model(inputs=best_model.input, outputs=penult_layer)
embeddings = feat_extractor.predict(X_te)

# 2) Choose a safe perplexity
n_samples = embeddings.shape[0]
perplexity = max(5, min(30, (n_samples // 3) - 1))
print(f"Using perplexity={perplexity} (n_samples={n_samples})")

# 3) t-SNE to 2D
tsne = TSNE(n_components=2, perplexity=perplexity, random_state=42)
proj = tsne.fit_transform(embeddings)

# 4) Ensure output folder exists
os.makedirs('model_plots', exist_ok=True)

# 5) Plot, colored by first attribute (e.g., 'gender')
fig, ax = plt.subplots(figsize=(6, 6))
for label in np.unique(y_te_int[0]):
    mask = (y_te_int[0] == label)
    ax.scatter(
        proj[mask, 0],
        proj[mask, 1],
        label=label_encoders[attrs[0]][label],
        alpha=0.6
    )

ax.legend(loc='best', fontsize='small')
ax.set_xlabel('t-SNE Dim 1')
ax.set_ylabel('t-SNE Dim 2')
ax.set_title(f"t-SNE of Penultimate Features (perplexity={perplexity})")
plt.tight_layout()

# 6) Save the figure
output_path = 'model_plots/tsne_penultimate_features.png'
fig.savefig(output_path, dpi=300, bbox_inches='tight')
plt.close(fig)

print(f"t-SNE plot saved to {output_path}")
plt.tight_layout()
plt.show()

157/157 ━━━━━━━━━━━━━━━━━━━━ 25s 155ms/step
Using perplexity=30 (n_samples=5000)
t-SNE plot saved to model_plots/tsne_penultimate_features.png


<Figure size 640x480 with 0 Axes>

In [24]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, precision_recall_curve, average_precision_score,
    roc_curve, auc, f1_score
)
from sklearn.manifold import TSNE
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model

# Ensure output folder exists
os.makedirs('model_plots', exist_ok=True)

sns.set(style="whitegrid")
palette = sns.color_palette("tab10", n_colors=len(attrs))
hist = histories[best_bs].history  # choose your best batch size
epochs_range = range(1, len(hist['loss']) + 1)

# ——— 1. Training & Validation Curves ———
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))
# Loss
ax1.plot(epochs_range, hist['loss'], label='Train Loss', color='C0')
ax1.plot(epochs_range, hist['val_loss'], label='Val Loss',   color='C1')
ax1.set_title('Total Loss vs. Epoch', fontsize=14)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend(loc='upper right')
# Mean Accuracy
mean_train = [
    np.mean([hist[f'{a}_accuracy'][i] for a in attrs])
    for i in range(len(epochs_range))
]
mean_val = [
    np.mean([hist[f'val_{a}_accuracy'][i] for a in attrs])
    for i in range(len(epochs_range))
]
ax2.plot(epochs_range, mean_train, label='Train Mean Acc', color='C2')
ax2.plot(epochs_range, mean_val,   label='Val Mean Acc',   color='C3')
ax2.set_title('Mean Tag Accuracy vs. Epoch', fontsize=14)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend(loc='upper right')
plt.tight_layout()
fig.savefig('model_plots/training_validation_curves.png', dpi=300, bbox_inches='tight')
plt.close(fig)

# ——— 2. Normalized Confusion Matrices ———
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
for i, attr in enumerate(attrs):
    ax = axes.flat[i]
    cm = confusion_matrix(y_te_int[i], np.argmax(y_pred[i], axis=1))
    cm_norm = cm.astype(float) / cm.sum(axis=1, keepdims=True)
    sns.heatmap(
        cm_norm, annot=True, fmt='.2f', cmap='Blues',
        xticklabels=label_encoders[attr],
        yticklabels=label_encoders[attr],
        cbar=False, ax=ax
    )
    ax.set_title(f'{attr} Confusion Matrix (Normalized)', fontsize=12)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')
plt.tight_layout()
fig.savefig('model_plots/normalized_confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.close(fig)

# ——— 3. Precision–Recall Curves ———
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
for i, attr in enumerate(attrs):
    ax = axes.flat[i]
    y_true = to_categorical(y_te_int[i], num_classes[attr])
    y_score = y_pred[i]
    for c in range(num_classes[attr]):
        prec, rec, _ = precision_recall_curve(y_true[:, c], y_score[:, c])
        ap = average_precision_score(y_true[:, c], y_score[:, c])
        ax.plot(rec, prec,
                label=f'{label_encoders[attr][c]} (AP={ap:.2f})',
                color=palette[c % len(palette)])
    ax.set_title(f'{attr} Precision–Recall', fontsize=12)
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.tight_layout()
fig.savefig('model_plots/precision_recall_curves.png', dpi=300, bbox_inches='tight')
plt.close(fig)

# ——— 4. ROC Curves & AUC ———
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
for i, attr in enumerate(attrs):
    ax = axes.flat[i]
    y_true = to_categorical(y_te_int[i], num_classes[attr])
    y_score = y_pred[i]
    for c in range(num_classes[attr]):
        fpr, tpr, _ = roc_curve(y_true[:, c], y_score[:, c])
        r_auc = auc(fpr, tpr)
        ax.plot(fpr, tpr,
                label=f'{label_encoders[attr][c]} (AUC={r_auc:.2f})',
                color=palette[c % len(palette)])
    ax.plot([0, 1], [0, 1], 'k--', linewidth=1)
    ax.set_title(f'{attr} ROC Curve', fontsize=12)
    ax.set_xlabel('FPR')
    ax.set_ylabel('TPR')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.tight_layout()
fig.savefig('model_plots/roc_curves.png', dpi=300, bbox_inches='tight')
plt.close(fig)

# ——— 5. F1-Score Bar Charts ———
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
for i, attr in enumerate(attrs):
    ax = axes.flat[i]
    y_pred_labels = np.argmax(y_pred[i], axis=1)
    f1s = f1_score(
        y_te_int[i], y_pred_labels,
        labels=list(range(num_classes[attr])),
        average=None, zero_division=0
    )
    sns.barplot(
        x=list(label_encoders[attr]), y=f1s,
        palette="tab10", ax=ax
    )
    ax.set_title(f'{attr} F1-Scores', fontsize=12)
    ax.set_ylim(0, 1)
    ax.set_ylabel('F1 Score')
    ax.set_xlabel('')
    for idx, val in enumerate(f1s):
        ax.text(idx, val + 0.02, f'{val:.2f}', ha='center', fontsize=9)
plt.tight_layout()
fig.savefig('model_plots/f1_score_barcharts.png', dpi=300, bbox_inches='tight')
plt.close(fig)

# ——— 6. t-SNE of Penultimate Features ———
penult = best_model.layers[-2].output
feat_extractor = Model(best_model.input, penult)
emb = feat_extractor.predict(X_te)
tsne = TSNE(n_components=2, random_state=42)
proj = tsne.fit_transform(emb)

fig, ax = plt.subplots(figsize=(6, 6))
for lbl in np.unique(y_te_int[0]):
    mask = (y_te_int[0] == lbl)
    ax.scatter(
        proj[mask, 0], proj[mask, 1],
        alpha=0.6, label=label_encoders[attrs[0]][lbl]
    )
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.set_title('t-SNE Projection of Penultimate Features', fontsize=14)
ax.set_xlabel('Dim 1')
ax.set_ylabel('Dim 2')
plt.tight_layout()
fig.savefig('model_plots/tsne_penultimate_features.png', dpi=300, bbox_inches='tight')
plt.close(fig)

print("All plots have been saved in the 'model_plots' directory.")
plt.tight_layout()
plt.show()

c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_ranking.py:980: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_ranking.py:980: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_ranking.py:980: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_ranking.py:980: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\patel\OneDrive\Desktop\FPR\tf-gpu-venv\Lib\site-packages\sklearn\metrics\_ranking.py:980: UserWarning: No positive class found in y_true, recall is set to one for all thre

157/157 ━━━━━━━━━━━━━━━━━━━━ 25s 156ms/step
All plots have been saved in the 'model_plots' directory.


<Figure size 640x480 with 0 Axes>

In [25]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Ensure output folder exists
os.makedirs('model_plots', exist_ok=True)

# ——— 1. Accuracy Bar Chart for All Batch Sizes ———
mean_acc_per_bs = {}
for bs in batch_sizes:
    h = histories[bs].history
    final_accs = [h[f'val_{a}_accuracy'][-1] for a in attrs]
    mean_acc_per_bs[bs] = np.mean(final_accs)

fig, ax = plt.subplots(figsize=(6, 4))
sns.barplot(
    x=list(mean_acc_per_bs.keys()),
    y=list(mean_acc_per_bs.values()),
    palette="tab10",
    ax=ax
)
ax.set_title("Mean Final Validation Accuracy by Batch Size", fontsize=14)
ax.set_xlabel("Batch Size", fontsize=12)
ax.set_ylabel("Mean Validation Accuracy", fontsize=12)
ax.set_ylim(0, 1)
for i, v in enumerate(mean_acc_per_bs.values()):
    ax.text(i, v + 0.02, f"{v:.2f}", ha='center', fontsize=10)
plt.tight_layout()

# Save accuracy bar chart
acc_bar_path = 'model_plots/mean_validation_accuracy_by_batch_size.png'
fig.savefig(acc_bar_path, dpi=300, bbox_inches='tight')
plt.close(fig)
print(f"Saved accuracy bar chart to {acc_bar_path}")

# ——— 2. Overall Confusion Matrix for All Attributes ———
# Build global labels and offsets
offsets = {}
cum = 0
global_labels = []
for a in attrs:
    offsets[a] = cum
    for cls in label_encoders[a]:
        global_labels.append(f"{a}_{cls}")
    cum += num_classes[a]
total_classes = cum

# Flatten true & pred across all attrs
y_true_global = []
y_pred_global = []
for i, a in enumerate(attrs):
    off = offsets[a]
    y_true_global.extend((np.array(y_te_int[i]) + off).tolist())
    y_pred_global.extend((np.argmax(y_pred[i], axis=1) + off).tolist())

cm_overall = confusion_matrix(
    y_true_global,
    y_pred_global,
    labels=np.arange(total_classes)
)

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(
    cm_overall,
    cmap="Blues",
    xticklabels=global_labels,
    yticklabels=global_labels,
    annot=False,
    cbar_kws={'shrink': 0.5},
    ax=ax
)
ax.set_title("Overall Confusion Matrix (All Attributes)", fontsize=14)
ax.set_xlabel("Predicted Label", fontsize=12)
ax.set_ylabel("True Label", fontsize=12)
plt.xticks(rotation=90, fontsize=8)
plt.yticks(rotation=0, fontsize=8)
plt.tight_layout()

# Save overall confusion matrix
cm_path = 'model_plots/overall_confusion_matrix_all_attributes.png'
fig.savefig(cm_path, dpi=300, bbox_inches='tight')
plt.close(fig)
print(f"Saved overall confusion matrix to {cm_path}")

plt.tight_layout()
plt.show()

Saved accuracy bar chart to model_plots/mean_validation_accuracy_by_batch_size.png


C:\Users\patel\AppData\Local\Temp\ipykernel_23512\2238784168.py:18: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(


Saved overall confusion matrix to model_plots/overall_confusion_matrix_all_attributes.png


<Figure size 640x480 with 0 Axes>